# Inspecting Elements of Q# Programs

## Classical Variables

In [1]:
open Microsoft.Quantum.Intrinsic;

/// # Summary
/// Demo of inspecting classical variables using the Message function.
operation InspectClassicalVariablesDemo(n : Int) : Unit {
    Message($"The value of the classical variable is {n}");
}

InspectClassicalVariablesDemo

In [2]:
%simulate InspectClassicalVariablesDemo n=5

The value of the classical variable is 5


()

## Quantum States

In [3]:
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Preparation;

/// # Summary
/// Demo of inspecting quantum states using the DumpMachine function.
operation InspectQuantumStateDemo() : Unit {
    use qs = Qubit[2];
    PrepareArbitraryStateCP([
            ComplexPolar(1., 0.),
            ComplexPolar(0., 0.),
            ComplexPolar(2., PI()/2.0),
            ComplexPolar(-2., 0.)
        ],
        LittleEndian(qs));
    DumpMachine();
    ResetAll(qs);
}

InspectQuantumStateDemo

In [4]:
%simulate InspectQuantumStateDemo

|0⟩	0.33333333333333326 + -1.3877787807814457E-17𝑖
|1⟩	0 + 0𝑖
|2⟩	-2.7755575615628914E-17 + 0.6666666666666667𝑖
|3⟩	-0.6666666666666667 + 2.7755575615628914E-17𝑖

()

Configuring the output of DumpMachine (Q# Jupyter Notebooks only)

In [5]:
%config dump.basisStateLabelingConvention="BitString"
%config dump.truncateSmallAmplitudes=true
%config dump.phaseDisplayStyle="None"
%simulate InspectQuantumStateDemo

|00⟩	0.33333333333333326 + -1.3877787807814457E-17𝑖
|01⟩	-2.7755575615628914E-17 + 0.6666666666666667𝑖
|11⟩	-0.6666666666666667 + 2.7755575615628914E-17𝑖

()

## Quantum Operations

In [6]:
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Intrinsic;

operation ControlledRxWrapper(
    qs : Qubit[],
    theta : Double
) : Unit is Adj + Ctl {
    Controlled Rx(Most(qs), (theta, Tail(qs)));
}

/// # Summary
/// Demo of inspecting quantum operations using the DumpOperation function.
operation InspectQuantumOperationDemo() : Unit {
    DumpOperation(2, ControlledRxWrapper(_, PI() / 2.0));
}

ControlledRxWrapper, InspectQuantumOperationDemo

In [7]:
%simulate InspectQuantumOperationDemo

Real:
[[1, 0, 0, 0], 
[0, 0.7071067811865476, 0, 0], 
[0, 0, 1, 0], 
[0, 0, 0, 0.7071067811865476]]
Imag:
[[0, 0, 0, 0], 
[0, 0, 0, -0.7071067811865476], 
[0, 0, 0, 0], 
[0, -0.7071067811865476, 0, 0]]

()

Switching the operation inspected from little endian (default) to big endian.

In [8]:
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Intrinsic;

operation ControlledRxWrapperBE(
    qs : Qubit[],
    theta : Double
) : Unit is Adj + Ctl {
    Controlled Rx(Rest(qs), (theta, Head(qs)));
}

/// # Summary
/// Demo of inspecting quantum operations using the DumpOperation function.
operation InspectQuantumOperationBEDemo() : Unit {
    DumpOperation(2, ControlledRxWrapperBE(_, PI() / 2.0));
}

ControlledRxWrapperBE, InspectQuantumOperationBEDemo

In [9]:
%simulate InspectQuantumOperationBEDemo

Real:
[[1.0000000000000002, 0, 0, 0], 
[0, 1.0000000000000002, 0, 0], 
[0, 0, 0.7071067811865477, 0], 
[0, 0, 0, 0.7071067811865477]]
Imag:
[[0, 0, 0, 0], 
[0, 0, 0, 0], 
[0, 0, 0, -0.7071067811865477], 
[0, 0, -0.7071067811865477, 0]]

()

## Program Structure and Evolution

In [10]:
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Intrinsic;

/// # Summary
/// A marking oracle for f(x) = sum of bits of x modulo 2.
operation SumOfBitsMarkingOracle(
    input : Qubit[],
    output : Qubit
) : Unit is Adj + Ctl {
    for q in input {
        CNOT(q, output);
    }
}


/// # Summary
/// Converts a marking oracle to a phase oracle using phase kickback trick.
operation MarkingAsPhaseOracle(markingOracle : (Qubit[], Qubit) => Unit, input : Qubit[]) : Unit {
    use target = Qubit();
    within {
        X(target);
        H(target);
    } apply {
        markingOracle(input, target);
    }
}


/// # Summary
/// Demo of inspecting quantum operations using the DumpOperation function.
operation RunDeutschJozsaAlgorithm(N : Int) : Bool {
    mutable isConstant = true;

    use qs = Qubit[2];
    ApplyToEach(H, qs);
    
    let oracle = MarkingAsPhaseOracle(SumOfBitsMarkingOracle, _);
    oracle(qs);

    ApplyToEach(H, qs);

    for q in qs {
        if M(q) == One {
            set isConstant = false;
        }
    }
    
    return isConstant;
}

SumOfBitsMarkingOracle, MarkingAsPhaseOracle, RunDeutschJozsaAlgorithm

In [11]:
%simulate RunDeutschJozsaAlgorithm N=2

False

In [12]:
%trace RunDeutschJozsaAlgorithm N=2

Microsoft.Quantum.IQSharp.Jupyter.DisplayableHtmlElement

In [ ]:
%debug RunDeutschJozsaAlgorithm N=2